In [1]:
from autoscraper import AutoScraper
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
Query = "Mobile"

BaseURL = "https://www.reliancedigital.in/smartphones/c/S101711?searchQuery="+Query+":relevance"

In [3]:
Page_Number = 50

listing_urls = [BaseURL]

for i in range(1, Page_Number + 1):
    listing_url = f"{BaseURL}&page={i}"
    listing_urls.append(listing_url)

# Now, listing_urls contains the URLs for all the pages
for url in listing_urls:
    print(url)

https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=1
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=2
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=3
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=4
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=5
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=6
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=7
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=8
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=9
https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=10
https://www.reliancedigital

In [4]:
#Function to parse the html page using Beautiful Soup......
def parse_html(url):
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
    return soup


In [ ]:
#Just ignore as ofnow....
URLS = "https://www.reliancedigital.in/smartphones/c/S101711?searchQuery=Mobile:relevance&page=1"

#URLS= "https://www.reliancedigital.in/page/samsung-mobiles"

response = requests.get(URLS)
soup = BeautifulSoup(response.text, 'html.parser')
html_string = soup.prettify()
html_string

In [5]:
original_url = "https://www.reliancedigital.in"
links = []
for url in listing_urls:
    soup = parse_html(url)
    a = soup.find_all("div",class_="slick-slide slick-active")
    for items in a:
            rest_link = items.find('a')['href']
            links.append(rest_link)

filtered_links = [original_url+link for link in links if not link.startswith('/page')]



In [6]:
filtered_links.

['https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332',
 'https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801',
 'https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832',
 'https://www.reliancedigital.in/samsung-galaxy-m13-5g-64-gb-4-gb-ram-midnight-blue-mobile-phone/p/493664825',
 'https://www.reliancedigital.in/vivo-y02t-4-gb-ram-64-gb-sunset-gold-mobile-phone/p/493838075',
 'https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801',
 'https://www.reliancedigital.in/xiaomi-redmi-a2-plus-64-gb-4-gb-ram-green-mobile-phone/p/493837926',
 'https://www.reliancedigital.in/samsung-galaxy-m13-5g-64-gb-4-gb-ram-midnight-blue-mobile-phone/p/493664825',
 'https://www.reliancedigital.in/apple-iphone-13-128-gb-midnight-black-/p/491997699',
 'https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332',
 'https://www.reliancedigi

In [7]:
len(filtered_links)

612

In [92]:
reviews = []

for urls in filtered_links:
    soup = parse_html(urls)
    
    product_title_element = soup.find('div', class_='bc__breadcrumbMain')
    mobile_name = product_title_element.find("span", id="RIL_BreadCrumb_3")
    mobile_names = mobile_name.text.strip()

    review_div = soup.find_all("div", class_="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh")
    
    # Initialize the product_reviews dictionary outside of the loop
    product_reviews = {
        'Product Title': mobile_names,
        'Reviews': []
    }
    
    # Loop through reviews for the current URL
    for r in review_div:
        review_dict = {}

        review_name = r.find("span", class_="TextWeb__Text-sc-1cyx778-0 itaoSP")
        review_dict['Review Name'] = review_name.text.strip() if review_name else None

        review_certified = r.find("span", class_="TextWeb__Text-sc-1cyx778-0 hNgtag")
        review_dict['Review Certified'] = review_certified.text.strip() if review_certified else None

        reviewer_location = r.find("div", class_="TextWeb__Text-sc-1cyx778-0 gqpvtQ")
        desc = reviewer_location.get_text()
        if desc:
            parts = [part.strip() for part in desc.split(',')]
            review_dict['Location'] = parts[0]
            review_dict['Date of Review'] = parts[1]
        else:
            review_dict['Location'] = None
            review_dict['Date of Review'] = None

        reviewer_title = r.find("span", class_="TextWeb__Text-sc-1cyx778-0 gHzvNl Block-sc-u1lygz-0 bsVFzL")
        review_dict['Reviewer Title'] = reviewer_title.text.strip() if reviewer_title else None

        reviewer_desc = r.find("div", class_="TextWeb__Text-sc-1cyx778-0 SingleReview__ReviewText-sc-1knvtv4-0 jSDjlD hjNCge")
        review_dict['Reviewer Description'] = reviewer_desc.text.strip() if reviewer_desc else None

        review_helpful = r.find("div", class_="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 hZaHav")
        review_dict['Review Helpfulness'] = review_helpful.text.strip() if review_helpful else None

        # Append the review data to the product_reviews['Reviews'] list
        product_reviews['Reviews'].append(review_dict)

    # Append the product_reviews dictionary to the reviews list only once
    reviews.append(product_reviews)

# Print the reviews list (containing data for all URLs)
for product_review in reviews:
    print(product_review)


https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 StarRating__StarRatingOuter-sc-1mfqajc-0 hABNCN iZYBRd"><i class="fa fa-star"></i><i class="fa fa-star"></i><i class="fa fa-star"></i><i class="fa fa-star"></i><i class="fa fa-star"></i></span><span cl

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

...

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/redmi-11-prime-64-gb-4-gb-ram-peppy-purple-mobile-phone/p/493178801
[]
https://www.reliancedigital.in/oppo-a78-5g-128-gb-8-gb-ram-glowing-blue-mobile-phone/p/493664832
[<div class="TextWeb__Text-sc-1cyx778-0 eUJwIX Block-sc-u1lygz-0 bKNhnh"><div class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 fkHCfb"><span class="TextWeb__Text-sc-1cyx778-0 itaoSP">RAVI YADAV</span><span class="TextWeb__Text-sc-1cyx778-0 jSDjlD Block-sc-u1lygz-0 bsVFzL"><span class="TextWeb__Text-sc-1cyx778-0 hNgtag"><i class="fa fa-check-circle"></i> <!-- -->Certified Buyer</span></span></div><div class="TextWeb__Te

https://www.reliancedigital.in/apple-iphone-15-plus-256-gb-blue/p/493839332
[]
https://www.reliancedigital.in/oneplus-nord-ce3-5g-256gb-12gb-ram-aqua-surge-mobile-phone/p/493838774
[]
https://www.reliancedigital.in/oneplus-nord-3-5g-256-gb-16-gb-ram-misty-green-mobile-phone/p/493838381
[]
{'Product Title': 'Apple iPhone 15 Plus 256 GB, Blue', 'Reviews': []}
{'Product Title': 'Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple Mobile Phone', 'Reviews': []}
{'Product Title': 'Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mobile Phone', 'Reviews': [{'Review Name': 'RAVI YADAV', 'Review Certified': 'Certified Buyer', 'Location': 'SURAT', 'Date of Review': '9/9/2023', 'Reviewer Title': 'Awsm', 'Reviewer Description': 'All gud at all the services to help', 'Review Helpfulness': None}, {'Review Name': 'Kiran Kadam', 'Review Certified': 'Certified Buyer', 'Location': '', 'Date of Review': '9/9/2023', 'Reviewer Title': 'Excellent', 'Reviewer Description': 'Excellent  service   .....   .............', '

In [94]:
len(reviews)

612

In [108]:
reviews

[{'Product Title': 'Apple iPhone 15 Plus 256 GB, Blue', 'Reviews': []},
 {'Product Title': 'Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple Mobile Phone',
  'Reviews': []},
 {'Product Title': 'Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mobile Phone',
  'Reviews': [{'Review Name': 'RAVI YADAV',
    'Review Certified': 'Certified Buyer',
    'Location': 'SURAT',
    'Date of Review': '9/9/2023',
    'Reviewer Title': 'Awsm',
    'Reviewer Description': 'All gud at all the services to help',
    'Review Helpfulness': None},
   {'Review Name': 'Kiran Kadam',
    'Review Certified': 'Certified Buyer',
    'Location': '',
    'Date of Review': '9/9/2023',
    'Reviewer Title': 'Excellent',
    'Reviewer Description': 'Excellent  service   .....   .............',
    'Review Helpfulness': None},
   {'Review Name': 'VINEET PARIDA',
    'Review Certified': 'Certified Buyer',
    'Location': 'Patiala',
    'Date of Review': '9/6/2023',
    'Reviewer Title': 'Best packaging.',
    'Reviewer Descrip

In [106]:
Product_Name = []
product_data_list = []

for urls in filtered_links:
    #urls = "https://www.reliancedigital.in/samsung-galaxy-m13-5g-64-gb-4-gb-ram-midnight-blue-mobile-phone/p/493664825"
    soup= parse_html(urls)
    product_title_element = soup.find('div', class_='bc__breadcrumbMain')
    mobile_name = product_title_element.find("span",id="RIL_BreadCrumb_3")
    discounts = soup.find("div",class_="TextWeb__Text-sc-1cyx778-0 hWKBsp Block-sc-u1lygz-0 iPHchL")
    price_types = soup.find("ul",class_="pdp__priceSection__priceList")
    price_items = soup.find_all("li", class_="pdp__priceSection__priceListText")

    # Initialize variables for prices and discounts
    offer_element = None
    mrp_element = None
    deal_element = None
    discount = None

    # Loop through the price items and extract the prices
    for item in price_items:
        text = item.get_text()
        if "Offer Price:" in text:
            offer_price = item.find("span", class_="TextWeb__Text-sc-1cyx778-0 bNdnUu")
            if offer_price:
                offer_element = offer_price.get_text()
                
        elif "MRP:" in text:
            mrp_price = item.find("span", class_="TextWeb__Text-sc-1cyx778-0 bNdnUu")
            if mrp_price:
                mrp_element = mrp_price.get_text()
        elif "Deal Price:" in text:
            deal_price = item.find("span", class_="TextWeb__Text-sc-1cyx778-0 cJQfDP")
            if deal_price:
                deal_element = deal_price.get_text()

    if discounts:
        discount = discounts.text.strip()
        discount = discount.replace("You Save: ", "")

    product_data = {
        'Product Name': mobile_name.text.strip() if mobile_name else None,
        'Deal Price': deal_element,
        'Offer Price': offer_element,
        'MRP': mrp_element,
        'Discounts': discount
    }
    product_data_list.append(product_data)


In [107]:
product_data_list

[{'Product Name': 'Apple iPhone 15 Plus 256 GB, Blue',
  'Deal Price': None,
  'Offer Price': None,
  'MRP': None,
  'Discounts': None},
 {'Product Name': 'Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple Mobile Phone',
  'Deal Price': '₹8,699.00',
  'Offer Price': '₹10,999.00',
  'MRP': '₹14,999.00',
  'Discounts': '42%(₹6,300)'},
 {'Product Name': 'Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mobile Phone',
  'Deal Price': None,
  'Offer Price': None,
  'MRP': '₹21,999.00',
  'Discounts': '14%(₹3,000)'},
 {'Product Name': 'Samsung Galaxy M13 5G 64 GB, 4 GB RAM, Midnight Blue, Mobile Phone',
  'Deal Price': '₹11,499.00',
  'Offer Price': '₹13,999.00',
  'MRP': '₹16,999.00',
  'Discounts': '32%(₹5,500)'},
 {'Product Name': 'Vivo Y02T 4 GB RAM 64 GB Sunset Gold Mobile Phone',
  'Deal Price': None,
  'Offer Price': None,
  'MRP': '₹15,999.00',
  'Discounts': '44%(₹7,000)'},
 {'Product Name': 'Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple Mobile Phone',
  'Deal Price': '₹8,699.00',
  'Offer Pri

In [98]:
product_data_list

[{'Product Name': <span id="RIL_BreadCrumb_3">Apple iPhone 15 Plus 256 GB, Blue<!-- --> </span>,
  'DealPrice': '₹99,900.00',
  'Offer_Price': None,
  'MRP': None,
  'Discounts': None},
 {'Product Name': <span id="RIL_BreadCrumb_3">Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple Mobile Phone<!-- --> </span>,
  'DealPrice': '₹14,999.00',
  'Offer_Price': None,
  'MRP': None,
  'Discounts': <div class="TextWeb__Text-sc-1cyx778-0 hWKBsp Block-sc-u1lygz-0 iPHchL"> <!-- -->You Save:<!-- --> <!-- -->42%(₹6,300)</div>},
 {'Product Name': <span id="RIL_BreadCrumb_3">Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mobile Phone<!-- --> </span>,
  'DealPrice': '₹21,999.00',
  'Offer_Price': None,
  'MRP': None,
  'Discounts': <div class="TextWeb__Text-sc-1cyx778-0 hWKBsp Block-sc-u1lygz-0 iPHchL"> <!-- -->You Save:<!-- --> <!-- -->14%(₹3,000)</div>},
 {'Product Name': <span id="RIL_BreadCrumb_3">Samsung Galaxy M13 5G 64 GB, 4 GB RAM, Midnight Blue, Mobile Phone<!-- --> </span>,
  'DealPrice': '₹16,999.0

In [13]:
#to find specifications :- 
all_specifications = []

for urls in filtered_links:
    soup = parse_html(urls)
    product_title_element = soup.find('div', class_='bc__breadcrumbMain')
    mobile_name = product_title_element.find("span",id="RIL_BreadCrumb_3")
    mobile_names = mobile_name.text.strip()
    specificiations_name = soup.find_all("div",class_="pdp__tab-info__list__name blk__sm__6 blk__xs__6")
    specificiations_value = soup.find_all("div",class_="pdp__tab-info__list__value blk__sm__6 blk__xs__6")

    specifications_dict = {}
    for name, value in zip(specificiations_name, specificiations_value):
                name_text = name.text.strip()
                value_text = value.text.strip()
                specifications_dict[name_text] = value_text
    specifications_dict["Mobile Name"] = mobile_names
    
    all_specifications.append(specifications_dict)

In [14]:
len(all_specifications)

612

In [15]:
all_specifications

[{'Colour': 'Blue',
  'Sub-brand': 'iPhone',
  'Model': '15 Plus',
  'Brand': 'Apple',
  'Internal Storage': '256 GB',
  'Processor': 'A16 Bionic Chip,6-core CPU,5-core GPU,16-core Neural Engine',
  'Operating System': 'iOS 17',
  'Operating System Type': 'iOS',
  'SIM Type': 'Dual SIM',
  '4G': 'Yes - LTE',
  '5G': 'Yes',
  'Battery Run Time': 'Video playback - Up to 26 hours\nVideo playback (streamed) - Up to 20 hours\nAudio playback - Up to 100 hours',
  'Quick Charge': 'Yes',
  'Bluetooth Version': 'v5.3',
  'Bluetooth': 'Yes',
  'Water Resistant': 'Yes',
  'Sensors': 'Barometer, Proximity, Accelerometer, Ambient Light',
  'Selfie Camera': '12 MP',
  'Screen Resolution': '2796 x 1290 px',
  'Screen Size (Diagonal)': '17.02 cm (6.7 inch)',
  'Rear Camera': '48 MP, 12 MP',
  'Display Type': 'Super Retina XDR',
  'Audio Formats': 'AAC, MP3, FLAC',
  'Video Formats': 'HEVC, H.264',
  'Features': 'Screen Resolution - 2796 x 1290-pixel resolution at 460 ppi\nEnabled by TrueDepth front ca

In [ ]:
#To get the customer reviews Dataframe......

In [114]:
reviewsdf = pd.DataFrame(reviews)
final_reviews = pd.json_normalize(reviewsdf.to_dict(orient='records'), 'Reviews', ['Product Title'])

column_order = [
    'Product Title',
    'Review Name',
    'Review Certified',
    'Location',
    'Date of Review',
    'Reviewer Title',
    'Reviewer Description',
    'Review Helpfulness']

final_reviews = final_reviews[column_order]

final_reviews

,Product Title,Review Name,Review Certified,Location,Date of Review,Reviewer Title,Reviewer Description,Review Helpfulness
0,"Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mo...",RAVI YADAV,Certified Buyer,SURAT,9/9/2023,Awsm,All gud at all the services to help,None
1,"Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mo...",Kiran Kadam,Certified Buyer,,9/9/2023,Excellent,Excellent service ..... .............,None
2,"Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mo...",VINEET PARIDA,Certified Buyer,Patiala,9/6/2023,Best packaging.,Product with best packaging and cover.,None
3,"Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mo...",Amit Kumar,Certified Buyer,,8/30/2023,Awesome 👍,Awesome experience shoping experience,13 people found this helpful
4,"Apple iPhone 13 128 GB, Midnight (Black)",Suman Das,Certified Buyer,Kolkata,8/11/2023,Worried about to buy iPhone from Reliance Digi...,"Initially, I was worried to buy from Reliance ...",174 people found this helpful
...,...,...,...,...,...,...,...,...
654,"Apple iPhone 13 128 GB, Midnight (Black)",Suman Das,Certified Buyer,Kolkata,8/11/2023,Worried about to buy iPhone from Reliance Digi...,"Initially, I was worried to buy from Reliance ...",174 people found this helpful
655,"Apple iPhone 13 128 GB, Midnight (Black)",Lebaku Maneeshwar Reddy,Certified Buyer,Cuddapah,5/31/2023,iPhone 13,No regrets.,One person found this helpful
656,"Apple iPhone 13 128 GB, Midnight (Black)",Harshit Manwani,Certified Buyer,,5/30/2023,iPhone 13,Great service by RD. the phone is fantastic.,None
657,"Apple iPhone 13 128 GB, Midnight (Black)",Sarvesh Tripathi,Certified Buyer,Lucknow,4/26/2023,Very good product,Nice service and nice product also 👍👍😊,2 people found this helpful


In [117]:
#To get the product data......

product_info =pd.DataFrame(product_data_list)

product_info.drop_duplicates

product_info

,Product Name,Deal Price,Offer Price,MRP,Discounts
0,"Apple iPhone 15 Plus 256 GB, Blue",None,None,None,None
1,"Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple M...","₹8,699.00","₹10,999.00","₹14,999.00","42%(₹6,300)"
2,"Oppo A78 5G 128 GB, 8 GB RAM, Glowing Blue, Mo...",None,None,"₹21,999.00","14%(₹3,000)"
3,"Samsung Galaxy M13 5G 64 GB, 4 GB RAM, Midnigh...","₹11,499.00","₹13,999.00","₹16,999.00","32%(₹5,500)"
4,Vivo Y02T 4 GB RAM 64 GB Sunset Gold Mobile Phone,None,None,"₹15,999.00","44%(₹7,000)"
...,...,...,...,...,...
607,"Samsung Galaxy M13 5G 64 GB, 4 GB RAM, Midnigh...","₹11,499.00","₹13,999.00","₹16,999.00","32%(₹5,500)"
608,"Apple iPhone 13 128 GB, Midnight (Black)","₹53,900.00","₹59,900.00","₹59,900.00","10%(₹6,000)"
609,"Apple iPhone 15 Plus 256 GB, Blue",None,None,None,None
610,"OnePlus Nord CE3 5G 256GB, 12GB RAM, Aqua Surg...",None,None,None,None


In [122]:
specifications_df = pd.DataFrame(all_specifications)

column_order = ['Mobile Name','Colour', 'Sub-brand', 'Model', 'Brand', 'Internal Storage',
       'Processor', 'Operating System', 'Operating System Type', 'SIM Type',
       '4G', '5G', 'Battery Run Time', 'Quick Charge', 'Bluetooth Version',
       'Bluetooth', 'Water Resistant', 'Sensors', 'Selfie Camera',
       'Screen Resolution', 'Screen Size (Diagonal)', 'Rear Camera',
       'Display Type', 'Audio Formats', 'Video Formats', 'Features',
       'Wireless Charging Support', 'In The Box', 'Warranty',
       'Customer care address', 'Customer care Phone', 'Customer care email',
       'Country of origin', 'Name and address of Packer',
       'Name and address of Manufacturer', 'Name and address of Importer',
       'Name of Seller',
       'Month and Year of Commodity First Manufactured/Imported/Packed',
       'BIS Model Number', 'Commodity name', 'Item Width', 'Item Height',
       'Item Length', 'Net Quantity', 'Net Weight',
       'Name and address of Marketed By','Storage Type', 'USB',
       'OTG Support', 'Battery Capacity', 'Audio Jack', 'Cellular Technology',
       'Fingerprint Recognition', 'Memory (RAM)', 'Series', 'Talk Time',
       'EDGE', 'Microphone']

specifications_df = specifications_df[column_order]

specifications_df.head(2)

,Mobile Name,Colour,Sub-brand,Model,Brand,Internal Storage,Processor,Operating System,Operating System Type,SIM Type,...,OTG Support,Battery Capacity,Audio Jack,Cellular Technology,Fingerprint Recognition,Memory (RAM),Series,Talk Time,EDGE,Microphone
0,"Apple iPhone 15 Plus 256 GB, Blue",Blue,iPhone,15 Plus,Apple,256 GB,"A16 Bionic Chip,6-core CPU,5-core GPU,16-core ...",iOS 17,iOS,Dual SIM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Redmi 11 Prime 64 GB, 4 GB RAM, Peppy Purple M...",Peppy Purple,Redmi,11 Prime,Xiaomi,64 GB,MediaTeK Helio G99 based on 6nm Processor,Android 12,Android,Dual SIM,...,Yes,5000 mAh,3.5 mm,GSM + WCDMA + LTE,Yes,4 GB,NaN,NaN,NaN,NaN


In [124]:
merged_df = product_info.merge(specifications_df, left_on='Product Name',right_on="Mobile Name" ,how='inner') 

In [129]:
merged_df.drop(columns=['Product Name'], inplace=True)

In [130]:
merged_df.count()

Deal Price      23460
Offer Price     23460
MRP             27948
Discounts       27948
Mobile Name     46818
                ...  
Memory (RAM)    33813
Series          18207
Talk Time       10404
EDGE             2601
Microphone       2601
Length: 62, dtype: int64

In [135]:
merged_df["Microphone"].unique()

array([nan, 'Yes'], dtype=object)

In [131]:
excel_file_path = 'C:\\Users\\Admin\\Desktop\\Solvathon\\reliance_product.xlsx'
merged_df.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to C:\Users\Admin\Desktop\Solvathon\reliance_product.xlsx


In [132]:
excel_file_path = 'C:\\Users\\Admin\\Desktop\\Solvathon\\reliance_mobile_reviews.xlsx'
final_reviews.to_excel(excel_file_path, index=False)

print(f'DataFrame saved to {excel_file_path}')

DataFrame saved to C:\Users\Admin\Desktop\Solvathon\reliance_mobile_reviews.xlsx
